In [1]:
from trino.dbapi import connect


# Step 1: Connect to Trino
conn = connect(
    host='localhost',
    port=8080,
    user='admin'
)
cursor = conn.cursor()

create processed iceberg tables 

In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.roles (
  id                    VARCHAR,
  id_user               VARCHAR,
  role                  VARCHAR,
  parent_id             VARCHAR,
  first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
  ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
  transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
  source_system VARCHAR COMMENT 'Source system from which the  record originated',
  is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
  comment VARCHAR COMMENT 'notes about invalidity of the record '        
)
WITH (
  format = 'PARQUET', 
  location = 's3://asel/processed/roles'   
  
)"""
)
print("table roles created !")

table roles created !


In [ ]:
print("=== Verifying  table ===")
cursor.execute("SELECT * FROM iceberg.processed.roles LIMIT 30")
rows = cursor.fetchall()

for row in rows:
    print(row)


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.activation (
  id                  VARCHAR COMMENT 'Unique activation record identifier',
  sim_id              VARCHAR COMMENT 'SIM card unique identifier',
  agent_id            VARCHAR COMMENT 'Identifier of the agent who processed the activation',
  customer_id         VARCHAR COMMENT 'Identifier of the customer who activated the SIM',
  mvno_id             VARCHAR COMMENT 'Mobile Virtual Network Operator identifier',
  createdAt           TIMESTAMP(6) COMMENT 'Activation timestamp (millisecond precision)',
  shopName            VARCHAR COMMENT 'Name of the retail location where activation occurred',
  first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
  ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
  transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
  source_system VARCHAR COMMENT 'Source system from which the  record originated',
  is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
  comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Activation record table for SIM activations, with linkage to customers, agents, and MVNOs'
WITH (
  format = 'PARQUET',
  partitioning = ARRAY[
    'year(createdAt)',
    'month(createdAt)',
    'day(createdAt)',
    'identity(shopName)'
  ],
  location = 's3://asel/processed/activation'
)
"""
)
print("table activation created !")



table activation created !


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.bundle_price_history (
    bundleid VARCHAR COMMENT 'Unique identifier of the bundle',
    price DOUBLE COMMENT 'Price of the bundle during the specified time range',
    start_date TIMESTAMP(6) COMMENT 'Start of the time range during which this price was valid (used for historical lookups)',
    end_date TIMESTAMP(6) COMMENT 'End of the time range during which this price was valid (used for historical lookups)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
               
)
COMMENT 'Historical pricing table for bundles, capturing price changes over time using validity periods'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/bundle_price_history'
)
""")
print("table bundle_price_history created !")

table bundle_price_history created !


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.bundles (
    id VARCHAR COMMENT 'Unique internal identifier for the record',
    name VARCHAR COMMENT 'Human-readable name of the bundle',
    bundleid VARCHAR COMMENT 'Business identifier for the bundle (shared across different versions or prices)',
    data_amount_gb DOUBLE COMMENT 'Amount of data included in the bundle, in gigabytes',
    voice_amount_minutes DOUBLE COMMENT 'Amount of voice call minutes included in the bundle',
    sms_amount INTEGER COMMENT 'Number of SMS messages included in the bundle',
    validity_days INTEGER COMMENT 'Number of days the bundle remains active once purchased',
    createdat TIMESTAMP(6) COMMENT 'Timestamp when the bundle definition was created (system ingestion or publishing date)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Table storing metadata and content definitions of asel bundles (data, voice, SMS, validity) to get the price of the bundle Join the bundle_price_history table with bundleid '
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/bundles'
)
""")
print("table bundles created !")

table bundles created !


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.customers (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the customer record',
    
    DOB TIMESTAMP(6) COMMENT 'Date of birth of the customer (nullable)',
    POB VARCHAR COMMENT 'Place of birth of the customer (nullable)',
    address VARCHAR COMMENT 'Postal address of the customer (nullable)',
    
    arta_id DOUBLE COMMENT 'Internal ARTA system identifier (nullable)',
    
    cin VARCHAR COMMENT 'National identity card number (nullable)',
    cin_recto_path VARCHAR COMMENT 'File path to front side scan of the CIN (nullable)',
    cin_verso_path VARCHAR COMMENT 'File path to back side scan of the CIN (nullable)',
    
    city VARCHAR COMMENT 'City of residence (nullable)',
    
    creation_date TIMESTAMP(6) COMMENT 'Timestamp when the customer record was created (nullable)',
    
    email VARCHAR COMMENT 'Customer email address (nullable)',
    first_name VARCHAR COMMENT 'Customer first name (nullable)',
    gender VARCHAR COMMENT 'Gender of the customer (nullable) male or female',
    
    issue_date TIMESTAMP(6) COMMENT 'Timestamp of identity document issuance (nullable)',
    
    job VARCHAR COMMENT 'Occupation or job title of the customer (nullable)',
    last_name VARCHAR COMMENT 'Customer last name (nullable)',
    
    mvno_id DOUBLE COMMENT 'ID of the MVNO (Mobile Virtual Network Operator) associated with the customer (nullable)',
    
    passport VARCHAR COMMENT 'Passport number (nullable)',
    passport_path VARCHAR COMMENT 'File path to scanned passport copy (nullable)',
    
    postal_code VARCHAR COMMENT 'Postal or ZIP code (nullable)',
    region VARCHAR COMMENT 'Administrative region of residence (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Customer master data table containing personal identification and contact details'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/customers',
    partitioning = ARRAY['bucket(id, 16)']
)
""")
print("table customers created !")

table customers created !


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.mvnos (
    id VARCHAR COMMENT 'Required unique identifier for the MVNO record',
    name VARCHAR COMMENT 'Name of the Mobile Virtual Network Operator ',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'MVNO master data table containing basic information about the mobile virtual network operator'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/mvnos'
)
""")
print("table mvno created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.paiement (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the payment record',
    
    commercial_id VARCHAR COMMENT 'Identifier of the commercial agent (nullable)',
    franchise_id VARCHAR COMMENT 'Identifier of the franchise (nullable)',
    transaction_id VARCHAR COMMENT 'Identifier of the transaction associated with the payment (nullable)',
    
    amount DOUBLE COMMENT 'Monetary amount of the payment (nullable)',
    
    created_at TIMESTAMP(6) COMMENT 'Timestamp when the payment was created (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Paiement transaction data containing reference IDs and payment amount'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/paiement',
    partitioning = ARRAY[
    'year(created_at)',
    'month(created_at)',
    'day(created_at)'
  ]
)
""")
print("table paiement created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.portability (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the portability record',
    
    sim_id VARCHAR COMMENT 'Identifier of the SIM card involved in the portability (nullable)',
    created_at TIMESTAMP(6) COMMENT 'Timestamp when the portability request was created (nullable)',
    agent_id VARCHAR COMMENT 'Identifier of the agent who handled the request (nullable)',
    
    old_number VARCHAR COMMENT 'Customer\'s previous phone number before portability (nullable)',
    code_rio VARCHAR COMMENT 'RIO (Operator Identity Statement) code required for portability (nullable)',
    current_operator VARCHAR COMMENT 'Current mobile network operator of the number (nullable)',
    
    customer_id VARCHAR COMMENT 'Identifier of the customer requesting the portability (nullable)',
    mvno_id VARCHAR COMMENT 'Identifier of the MVNO associated with the portability request (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Portability request data containing SIM, customer, and operator details'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/portability',
    partitioning = ARRAY[
    'year(created_at)',
    'month(created_at)',
    'day(created_at)'
  ]
)
""")
print("table portability created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.sallers (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the saller record',
    
    name VARCHAR COMMENT 'Full name of the saller (nullable)',
    role VARCHAR COMMENT 'Role or position of the saller in the commercial hierarchy (nullable)',
    
    parent_id VARCHAR COMMENT 'Identifier of the parent saller or manager (nullable)',
    shop_name VARCHAR COMMENT 'Name of the shop or point of sale (nullable)',
    
    joined_at TIMESTAMP(6) COMMENT 'Timestamp when the saller joined the organization (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Saller master data table containing commercial agent identity, role, and affiliation'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/sallers'
)
""")
print("table saller created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.shops (
    _id VARCHAR COMMENT 'Unique identifier of the shop record (nullable)',
    
    name VARCHAR COMMENT 'Name of the shop (nullable)',
    owner VARCHAR COMMENT 'Name or ID of the shop owner (nullable)',
    reseller_id VARCHAR COMMENT 'Identifier of the reseller linked to the shop (nullable)',
    
    latitude DOUBLE COMMENT 'Geographical latitude of the shop location (nullable)',
    longitude DOUBLE COMMENT 'Geographical longitude of the shop location (nullable)',
    
    adresse VARCHAR COMMENT 'Physical address of the shop (nullable)',
    created_at VARCHAR COMMENT 'Creation timestamp as a string format (nullable)',

    phone VARCHAR COMMENT 'Contact phone number of the shop (nullable)',
    gouvernorat VARCHAR COMMENT 'Administrative region or governorate where the shop is located (nullable)',
    status VARCHAR COMMENT 'Current operational status of the shop (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Shop master data containing location, ownership, and identification details'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/shops'
)
""")
print("table shoprecord created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.sims (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the SIM record',
    
    ICCID VARCHAR COMMENT 'Integrated Circuit Card Identifier of the SIM (nullable)',
    SN VARCHAR COMMENT 'Serial number of the SIM card (nullable)',
    MSISDN VARCHAR COMMENT 'Mobile Station International Subscriber Directory Number (phone number) (nullable)',
    IMSI VARCHAR COMMENT 'International Mobile Subscriber Identity (nullable)',
    
    mvno_id VARCHAR COMMENT 'Identifier of the MVNO associated with the SIM (nullable)',
    activation_date TIMESTAMP(6) COMMENT 'Timestamp when the SIM was activated (nullable)',
    
    customer_id VARCHAR COMMENT 'Identifier of the customer to whom the SIM is assigned (nullable)',
    partition VARCHAR COMMENT 'Logical partition information (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'SIM card master data containing identifiers, activation info, and associations'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/sims',
    partitioning = ARRAY['bucket(id, 16)','year(activation_date)','month(activation_date)']
)
""")
print("table sim created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.topupbundles (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the top-up bundle record',
    
    retailer_id VARCHAR COMMENT 'Identifier of the retailer who processed the top-up (nullable)',
    sim_id VARCHAR COMMENT 'Identifier of the SIM card receiving the top-up (nullable)',
    bundle_id VARCHAR COMMENT 'Identifier of the bundle activated in the top-up (nullable)',
    
    shopName VARCHAR COMMENT 'Name of the shop where the top-up occurred (nullable)',
    mvno_id VARCHAR COMMENT 'Identifier of the MVNO associated with the top-up (nullable)',
    
    createdAt TIMESTAMP(6) COMMENT 'Timestamp when the top-up was created (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Top-up bundle data including retailer, SIM, and bundle associations'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/topupbundles',
    partitioning = ARRAY[
    'year(createdAt)',
    'month(createdAt)',
    'day(createdAt)',
    'identity(shopName)'
  ]
)
""")
print("table topupbundles created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.topuplights (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the top-up light record',
    
    retailer_id VARCHAR COMMENT 'Identifier of the retailer who processed the top-up (nullable)',
    sim_id VARCHAR COMMENT 'Identifier of the SIM card receiving the top-up (nullable)',
    shopName VARCHAR COMMENT 'Name of the shop where the top-up occurred (nullable)',
    mvno_id VARCHAR COMMENT 'Identifier of the MVNO associated with the top-up (nullable)',
    
    createdAt TIMESTAMP(6) COMMENT 'Timestamp when the top-up was created (nullable)',
    amount DOUBLE COMMENT 'Monetary amount of the top-up (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Top-up light data including retailer, SIM, amount, and shop information'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/topuplights',
    partitioning = ARRAY[
    'year(createdAt)',
    'month(createdAt)',
    'day(createdAt)',
    'identity(shopName)']
)
""")
print("table topuplights created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.transactions (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the transaction record',
    
    sender_id VARCHAR COMMENT 'Identifier of the sender initiating the transaction (nullable)',
    recipient_id VARCHAR COMMENT 'Identifier of the recipient receiving the transaction (nullable)',
    
    transactionAmount DOUBLE COMMENT 'Monetary amount of the transaction (nullable)',
    createdAt TIMESTAMP(6) COMMENT 'Timestamp when the transaction was created (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Transaction data including sender, recipient, amount, and timestamp'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/transactions',
    partitioning = ARRAY[
    'year(createdAt)',
    'month(createdAt)',
    'day(createdAt)']
)
""")
print("table transactions created !")


In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.users (
    id VARCHAR NOT NULL COMMENT 'Required unique identifier for the user record',
    
    name VARCHAR COMMENT 'Full name of the user (nullable)',
    joined_at TIMESTAMP(6) COMMENT 'Timestamp when the user joined the system (nullable)',
    first_seen_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was first observed',
    ingestion_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was ingested into the data lake',
    transformation_date TIMESTAMP(6) COMMENT 'Timestamp when the  record was transformed in the lakehouse',
    source_system VARCHAR COMMENT 'Source system from which the  record originated',
    is_valid BOOLEAN DEFAULT TRUE COMMENT 'Flag indicating whether the record is valid',
    comment VARCHAR COMMENT 'notes about invalidity of the record '
)
COMMENT 'Processed user data including identity and join timestamp'
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/users'
)
""")
print("table processeduser created !")


now the final steps to processed zone will be finilised 

In [ ]:
cursor.execute("""INSERT INTO processed.sallers (id, name, role, parent_id, shop_name, joined_at)
SELECT
    r.id,         
    u.name,       
    r.role,        
    r.parent_id,  
    u.shop_name,  
    u.joined_at    
FROM
    processed.users u
JOIN
    processed.roles r
    ON u.id = r.id_user
WHERE
    r.id IS NOT NULL
    AND u.id IS NOT NULL
    AND r.id NOT IN (
        SELECT id FROM processed.sallers
    );
""")
print("alimentation_applied")

In [ ]:
cursor.execute("""CREATE TABLE iceberg.processed.sallers_updated
WITH (
    format = 'PARQUET',
    location = 's3://asel/processed/sallers_updated'
)
AS
WITH RECURSIVE ShopNameLookup AS (
    SELECT id, parent_id, shop_name, id AS original_id
    FROM iceberg.processed.sallers
    WHERE shop_name IS NULL

    UNION ALL

    SELECT p.id, p.parent_id, p.shop_name, sl.original_id
    FROM ShopNameLookup sl
    JOIN iceberg.processed.sallers p ON sl.parent_id = p.id
    WHERE sl.shop_name IS NULL
),
InheritedShopNames AS (
    SELECT original_id, MAX(shop_name) AS inherited_shop_name
    FROM ShopNameLookup
    GROUP BY original_id
)
SELECT
    s.id,
    s.name,
    s.role,
    s.parent_id,
    COALESCE(s.shop_name, i.inherited_shop_name) AS shop_name,
    s.joined_at,
    s.first_seen_date,
    s.ingestion_date,
    s.transformation_date,
    s.source_system,
    s.is_valid,
    s.comment
FROM iceberg.processed.sallers s
LEFT JOIN InheritedShopNames i ON s.id = i.original_id;

""")
print("table created")